In [61]:
from jupyter_plotly_dash import JupyterDash
import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "aacuserpwd"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
data = {}
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')
app.add_external_link = True

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Charles French | SNHU CS-340 Dashboard'))),
    html.Center(html.Img(id='customer_image',height='180', width='150',
                        src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 1},
            {'label': 'Mountain Rescue', 'value': 2},
            {'label': 'Disaster Rescue', 'value': 3},
            {'label': 'reset', 'value': 4}
        ],
        value= 4,
        labelStyle={'display': 'inline-block'}
        ),
        
    
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        
        data = df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #dcc.Graph(id='graph-id'),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])])
    
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-interactivity','data'),
               Output('datatable-interactivity','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        if(filter_type == 1):
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                           "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
                           "sex_upon_outcome":"Intact Female",
                           "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}}))
            
            
        elif(filter_type == 2):
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                              "breed":{"$in":["German Shephard","Alaskan Malamute","Old English Sheepdog",
                                       "Siberian Husky", "Rottweiler"]}, 
                              "sex_upon_outcome":"Intact Male",
                              "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}}))
        
        elif(filter_type == 3):
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                              "breed":{"$in":["Doberman Pinscher", "German Shephard","Golden Retriever","Bloodhound",
                                       "Rottweiler"]}, 
                              "sex_upon_outcome":"Intact Male",
                              "age_upon_outcome_in_weeks":{"$gte":20, "$lte":300}}))
            
        elif(filter_type == 4):
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data = df.to_dict('records')
        
        
        return (data,columns)
        #return dff.to_dict('records')



@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Percentage of breeds available')
            #dff.groupby(['breed']).sum().plot(kind='pie', y=dff.loc[:,'breed'].sum())
        )    
    ]
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = pd.DataFrame.from_dict(viewData)
    #dff = viewDF.loc[row]          
    lat = viewDF["location_lat"]
    long = viewDF["location_long"]
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat[0],long[0]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat[0],long[0]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ] 
    

app